In [1]:
from sympy import *
import sympy as sp
from sympy.polys.domains.finitefield import FiniteField

In [2]:
#un code pour un échelonnement de gauss en Z2
def equal_arrays(array1, array2):
  return all(array1[i] == array2[i] for i in range(len(array1)))

# Define a function to check if an array is all 0
def zero_array(array):
  return all(array[i] == 0 for i in range(len(array)))

# Define a function to delete repetitive arrays and all 0 arrays in a given matrix
def delete_arrays(matrix):
  # Get the number of rows and columns
  rows = len(matrix)
  cols = len(matrix[0])

  # Initialize a list to store the indices of the unique and nonzero rows
  unique_rows = []

  # Loop through the rows
  for i in range(rows):
    # Assume that the current row is unique and nonzero
    unique = True
    nonzero = True

    # Loop through the unique rows
    for j in unique_rows:
      # If the current row is equal to any of the unique rows, mark it as not unique
      if equal_arrays(matrix[i], matrix[j]):
        unique = False
        break

    # If the current row is unique, check if it is nonzero
    if unique:
      # If the current row is all 0, mark it as not nonzero
      if zero_array(matrix[i]):
        nonzero = False

    # If the current row is unique and nonzero, add its index to the list
    if unique and nonzero:
      unique_rows.append(i)

  # Create a new matrix with only the unique and nonzero rows
  new_matrix = [matrix[i] for i in unique_rows]

  # Return the new matrix
  return new_matrix

In [3]:
#la suite du code pour un échelonnement de gauss en Z2
# Define a function to perform XOR operation on two rows
def xor_rows(row1, row2):
  return [row1[i] ^ row2[i] for i in range(len(row1))]

# Define a function to swap two rows in a matrix
def swap_rows(matrix, i, j):
  matrix[i], matrix[j] = matrix[j], matrix[i]

# Define a function to perform Gaussian elimination over GF(2) on a matrix
def gauss_elim_gf2(matrix):
  # Get the number of rows and columns
  rows = len(matrix)
  cols = len(matrix[0])

  # Initialize the row and column indices
  r = 0
  c = 0

  # Loop through the columns
  while r < rows and c < cols:
    # Find a nonzero element in column c, starting from row r
    pivot = -1
    for i in range(r, rows):
      if matrix[i][c] == 1:
        pivot = i
        break

    # If a nonzero element is found, swap it with row r
    if pivot != -1:
      swap_rows(matrix, r, pivot)

      # Eliminate the coefficients below the pivot
      for i in range(r + 1, rows):
        if matrix[i][c] == 1:
          matrix[i] = xor_rows(matrix[i], matrix[r])

      # Increment the row index
      r += 1

    # Increment the column index
    c += 1

  # Return the matrix in row echelon form
  return matrix
y=[[1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0]]
gauss_elim_gf2(y)

[[1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1]]

In [4]:
#avoir tout les termes du systeme
def get_elements(x):
  y=[]
  for i in x:
    y=list(set(y)|set(i))
  return sorted(y, key=lambda x: len(x), reverse=True)

In [5]:
def remove_spaces(string):
    return string.replace(" ", "")
#simplifications des polynomes en Z2
def line_reduction(x):
  A="+p+p+2"#stabilzer pour eviter les erreurs, si la fonction n'a pas quelques choses a simplifier ou une constante
  expr = sp.parse_expr(x+A)
  x = (lambda p: sp.Poly(sp.ratsimpmodprime(p, [n**2-n for n in p.free_symbols]), modulus=2))(expr.as_expr())
  return [remove_spaces(i) for i in (str((x.args)[0])).split("+")]
line_reduction("x**10+y-x+2")

['y']

In [6]:
def verify(x,y):
  for i in x:
    if(i==y):
      return True
  return False
'''transform un system en matrice
x+x*y=0
y+1=0
[1,1,0,0]
[0,0,1,1]
et les variables:[x*y,x,y,1]'''
def system_to_matrix(x):
  y,s=get_elements(x),[]
  for i in x:
    x=[]
    for j in y:
      if(verify(i,j)):
        x+=[1]
      else:
        x+=[0]
    s+=[x]
  return s,y
system_to_matrix([["x","x*y"],["y","1"]])

([[1, 0, 0, 1], [0, 1, 1, 0]], ['x*y', 'y', '1', 'x'])

In [7]:
#detecter si 1=0
def error(matrix,y):
  i=len(y)-1
  while(0<=i and len(y[i])==1):
    if(y[i]=="1"):
      a=[0]*len(y)
      a[i]=1
      return any(a == list(j) for j in matrix)
    i-=1
  return False
  for i in matrix:
    if (i ==([0]*(look_for_the_one(y))+[1]+[0]*(len(y)-1-look_for_the_one(y)))):
      return True
  return False
#des fonctions pour mettres les fonctions principales en accord
def row_to_strings(x,y):
  z=[]
  for i in range(len(y)):
    if(x[i]==1):
      z+=[y[i]]
  return z
def row_to_line(x):
  s=x[0]
  for  i in range(1,len(x)):
    s+="+"+x[i]
  return s

In [8]:
#les cas d'existance d'une solution
def case_0(x,y):
  s=False
  for i in x:
    if(i.find(y)!=-1 ):
      if(len(i)==len(y)):
        s=True
      else:
        return False
  return s
def local(x,y):
  for i in x:
    if(remove_spaces(i)==y):
      return True
  return False
def case_1(x,y):
  s=False
  for i in x:
    if(i=="1"):
      s=True
    elif(not local(i.split("*"),y)):
      return False
  return s
def case_2(sol):
  if(sol.find("/")!=-1):
    a,b=line_reduction(sol[sol.find("/")+1:]),line_reduction(sol[:sol.find("/")]+" + 1")
    return a==b
  return False
def line_solve(x,y):
  if(case_0(x,y)):
    return str(solve(sympify(row_to_line(x)), sympify(y))[0]),[]
  elif(case_1(x,y)):
    sol=str(solve(sympify(row_to_line(x)), sympify(y))[0])
    return "1",[sol[sol.find("/")+1:]+" + 1"]
  elif(case_2(str(solve(sympify(row_to_line(x)), sympify(y))[0]))):
    sol=str(solve(sympify(row_to_line(x)), sympify(y))[0])
    return "0",[sol[:sol.find("/")]]
  else:
    #pas de solution
    return "nope","nope"

In [15]:
#si la variable existe on resoud sinon
def existance(x,y):
  for i in x:
    if(local(i.split("*"),y)):
      return True
  return False
def line_substitution(x,y,z):
  a=(sympify(row_to_line(x)))
  return str(a.subs(sympify(y),sympify(z)))
#touve une solution et substitue dans le systeme
def system_solve(x,y,A):
  for i in range(len(x)):
    for j in y:
      if( existance(x[i],j) and (line_solve(x[i],j))[0]!="nope"):
        a=x[i]
        A[j]=(line_solve(a,j))[0]
        x.pop(i)
        x=[line_substitution(k,j,(line_solve(a,j))[0]) for k in x]+(line_solve(a,j))[1]
        return A,x
  #le cas -1 donc il existe um multitude de solution
  print(update(A),[row_to_line(i) for i in x])
  #on determine une variable au hazard, on choisit 0 pour reduire les termes
  A[y[0]]="0"
  return A,[line_substitution(i,y[0],"0") for i in x]

In [10]:
#une fois les solutions donne on fonction d'autres variables et une variable determine, on substitue chaque variables et equations
def update(s):
  x = list(s.keys())
  y = list(s.values())
  if(x==[]):
    return s
  y[-1]=row_to_line(line_reduction(y[-1]))
  for i in range(1,len(x)):
    for j in reversed(range(len(x)-i)):
      y[j]= line_substitution(line_reduction(y[j]),x[len(x)-i],y[len(x)-i])
  for i in range(len(x)):
    s[x[i]]=row_to_line(line_reduction(y[i]))
  return s
#pour avoir les variables
def get_bases(x):
  y=[]
  for i in x:
    y=list(set(y)|set((sympify(row_to_line(i))).atoms(Symbol)))
  return [str(i) for i in y]
def Solver(system):
  A={}
  while(system!=[]):
    #print(A,system)
    system=[line_reduction(i) for i in system]
    #print(A,system)
    matrix_form,variables=system_to_matrix(system)[0],system_to_matrix(system)[1]
    #si c un systeme de 0
    if(set(variables)==set(['0'])):
      return update(A)
    #print(matrix_form,variables)
    #gaussian elemination
    reduced_matrix_form=delete_arrays(gauss_elim_gf2(matrix_form))
    #print(reduced_matrix_form)
    if(error(reduced_matrix_form,variables)):
      return("no solution")
    elif(reduced_matrix_form==[]):
      return update(A)
    else:
      reduced_system=[row_to_strings(i,variables) for i in reduced_matrix_form]
      #print(reduced_system)
      bases=get_bases(reduced_system)
      #print(bases)
      b=system_solve(reduced_system,bases,A)
      #print(b)
      A,system=b[0],b[1]
  return update(A)

In [11]:
def transform_or(x):
  y="("+"1+"+(x[0]).replace("¬", "1+")+")"
  for i in range(1,len(x)):
    y+="*"+"("+"1+"+x[i].replace("¬", "1+")+")"
  return y
#transform 3sat on systeme d'equation polyno en Z2
def to_system(x):
  return [transform_or(((remove_spaces(i))[1:-1]).split("∨")) for i in x.split("∧")]

In [12]:
s="(a OR b OR c) AND (NOT a OR NOT b OR NOT c) AND (a OR NOT b OR NOT c) AND (NOT a OR b OR NOT c) AND (NOT a OR NOT b OR c)"
s=s.replace("OR", "∨")
s=s.replace("NOT", "¬")
s=s.replace("AND", "∧")
s

'(a ∨ b ∨ c) ∧ (¬ a ∨ ¬ b ∨ ¬ c) ∧ (a ∨ ¬ b ∨ ¬ c) ∧ (¬ a ∨ b ∨ ¬ c) ∧ (¬ a ∨ ¬ b ∨ c)'

In [13]:
b="(not x5 or x2 or x4) and (x4 or not x6 or x1) and (not x1 or x8 or x3) and (not x8 or not x1 or not x9) and (not x7 or x6 or not x4) and (not x5 or x8 or x1) and (not x8 or not x7 or not x3) and (x4 or not x6 or x3) and (not x3 or x8 or x1) and (x9 or not x3 or x8) and (x2 or x3 or not x8) and (x8 or not x7 or not x1) and (not x1 or x8 or not x9) and (x2 or x4 or x5) and (x2 or x1 or x6)"

def transforers(s):
  s=s.replace("or", "∨")
  s=s.replace("not", "¬")
  return s.replace("and", "∧")
b=transforers(b)

In [16]:
x = "(x ∨ x ∨ y) ∧ (¬x ∨ ¬y ∨ ¬y) ∧ (¬x ∨ y ∨ y)"
Solver(['(1+1+x10)*(1+1+x16)*(1+1+x11)',
 '(1+x16)*(1+1+x3)*(1+x5)',
 '(1+x19)*(1+1+x20)*(1+x3)',
 '(1+1+x23)*(1+1+x6)*(1+x21)',
 '(1+1+x3)*(1+1+x15)*(1+x23)',
 '(1+x24)*(1+x19)*(1+x10)',
 '(1+x21)*(1+1+x10)*(1+1+x20)',
 '(1+x9)*(1+1+x19)*(1+x11)',
 '(1+x14)*(1+1+x8)*(1+x10)',
 '(1+x20)*(1+1+x9)*(1+1+x7)',
 '(1+1+x21)*(1+x1)*(1+1+x18)',
 '(1+1+x6)*(1+x11)*(1+x22)',
 '(1+1+x21)*(1+x18)*(1+x11)',
 '(1+x8)*(1+1+x4)*(1+1+x24)',
 '(1+1+x17)*(1+1+x15)*(1+x4)',
 '(1+1+x16)*(1+x2)*(1+x14)',
 '(1+1+x11)*(1+1+x9)*(1+x23)',
 '(1+1+x1)*(1+x20)*(1+x19)',
 '(1+1+x13)*(1+x20)*(1+1+x18)',
 '(1+x12)*(1+x24)*(1+1+x18)',
 '(1+1+x13)*(1+1+x18)*(1+x11)',
 '(1+x20)*(1+1+x12)*(1+1+x16)',
 '(1+1+x12)*(1+x22)*(1+x3)',
 '(1+x22)*(1+x10)*(1+1+x23)',
 '(1+x20)*(1+1+x16)*(1+x13)',
 '(1+x17)*(1+1+x8)*(1+x10)',
 '(1+x18)*(1+1+x19)*(1+x15)',
 '(1+1+x2)*(1+1+x5)*(1+1+x12)',
 '(1+x15)*(1+1+x24)*(1+1+x18)',
 '(1+x7)*(1+x3)*(1+1+x18)',
 '(1+1+x12)*(1+1+x20)*(1+1+x7)',
 '(1+1+x23)*(1+1+x16)*(1+x10)',
 '(1+1+x8)*(1+x7)*(1+1+x17)'])

{} ['x10*x20*x21+x10*x20', 'x10*x19*x24+x10*x19+x10*x24+x19*x24+x24+x19+x10+1', 'x10*x11*x16', 'x11*x13*x18+x13*x18', 'x13*x18*x20+x13*x18', 'x13*x16*x20+x16*x20+x13*x16+x16', 'x10*x16*x23+x16*x23', 'x10*x22*x23+x22*x23+x10*x23+x23', 'x12*x16*x20+x12*x16', 'x15*x18*x24+x18*x24', 'x11*x18*x21+x11*x21+x18*x21+x21', 'x12*x18*x24+x18*x24+x12*x18+x18', 'x15*x18*x19+x15*x19+x18*x19+x19', 'x14*x16*x2+x14*x16+x16*x2+x16', 'x10*x14*x8+x14*x8+x10*x8+x8', 'x1*x18*x21+x18*x21', 'x11*x23*x9+x11*x9', 'x19*x20*x3+x19*x20+x20*x3+x20', 'x11*x19*x9+x11*x19+x19*x9+x19', 'x21*x23*x6+x23*x6', 'x15*x17*x4+x15*x17', 'x12*x20*x7', 'x12*x22*x3+x12*x22+x12*x3+x12', 'x10*x17*x8+x17*x8+x10*x8+x8', 'x11*x22*x6+x22*x6+x11*x6+x6', 'x15*x23*x3+x15*x3', 'x1*x19*x20+x1*x20+x1*x19+x1', 'x20*x7*x9+x7*x9', 'x16*x3*x5+x16*x3+x3*x5+x3', 'x17*x7*x8+x17*x8', 'x24*x4*x8+x24*x4', 'x12*x2*x5', 'x18*x3*x7+x18*x7+x18*x3+x18']
{'x14': '0'} ['x10*x20*x21+x10*x20', 'x10*x19*x24+x10*x19+x10*x24+x19*x24+x24+x19+x10+1', 'x10*x11*x16', '

{'x14': '0',
 'x7': '0',
 'x12': '0',
 'x18': '0',
 'x16': '0',
 'x20': '0',
 'x23': '0',
 'x10': '0',
 'x8': '0',
 'x9': '0',
 'x22': '0',
 'x21': '0',
 'x17': '0',
 'x6': '0',
 'x24': '0',
 'x19': '1',
 'x11': '1',
 'x15': '1',
 'x3': '0'}

In [20]:
Solver(to_system(transforers(b)))

{} ['x2*x4*x5+x4*x5+x2*x5+x5', 'x3*x4*x6+x3*x6+x4*x6+x6', 'x1*x7*x8+x1*x7', 'x1*x2*x6+x2*x6+x1*x6+x1*x2+x6+x2+x1+1', 'x1*x8*x9+x1*x9', 'x1*x5*x8+x1*x5+x5*x8+x5', 'x2*x3*x8+x2*x8+x3*x8+x8', 'x1*x4*x6+x4*x6+x1*x6+x6', 'x4*x6*x7+x4*x7', 'x1*x3*x8+x1*x8+x1*x3+x1', 'x3*x8*x9+x3*x8+x3*x9+x3', 'x3*x7*x8', 'x1*x8+x3*x8+x3+x1', 'x1*x9', 'x2*x4+x4+x2+1']
{'x2': '0', 'x4': '1'} ['x1*x7*x8+x1*x7', 'x1*x3*x8+x1*x8+x1*x3+x1', 'x3*x8*x9+x3*x8+x3*x9+x3', 'x3*x7*x8', 'x1*x8*x9+x1*x9', 'x1*x5*x8+x5*x8+x1*x5+x5', 'x1*x8+x3*x8+x1+x3', 'x3*x8+x8', 'x1*x9', 'x1*x6+x6+x1+1', 'x6*x7+x7', '0']


{'x2': '0',
 'x4': '1',
 'x6': '0',
 'x7': '0',
 'x1': '1',
 'x3': '1',
 'x8': '1',
 'x9': '0'}